In [1]:
import numpy as np
import pandas as pd
import datetime as dt

In [78]:
df = pd.read_csv('C:/Users/ggibs/data/US_Accidents_Dec19.csv', parse_dates = ['Start_Time', 'End_Time'])  

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2974335 entries, 0 to 2974334
Data columns (total 49 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   ID                     object        
 1   Source                 object        
 2   TMC                    float64       
 3   Severity               int64         
 4   Start_Time             datetime64[ns]
 5   End_Time               datetime64[ns]
 6   Start_Lat              float64       
 7   Start_Lng              float64       
 8   End_Lat                float64       
 9   End_Lng                float64       
 10  Distance(mi)           float64       
 11  Description            object        
 12  Number                 float64       
 13  Street                 object        
 14  Side                   object        
 15  City                   object        
 16  County                 object        
 17  State                  object        
 18  Zipcode               

In [5]:
# % null
df.isna().sum()/2974335*100

ID                        0.000000
Source                    0.000000
TMC                      24.478446
Severity                  0.000000
Start_Time                0.000000
End_Time                  0.000000
Start_Lat                 0.000000
Start_Lng                 0.000000
End_Lat                  75.521554
End_Lng                  75.521554
Distance(mi)              0.000000
Description               0.000034
Number                   64.471722
Street                    0.000000
Side                      0.000000
City                      0.002791
County                    0.000000
State                     0.000000
Zipcode                   0.029586
Country                   0.000000
Timezone                  0.106343
Airport_Code              0.191337
Weather_Timestamp         1.234057
Temperature(F)            1.884892
Wind_Chill(F)            62.286965
Humidity(%)               1.989453
Pressure(in)              1.618580
Visibility(mi)            2.208595
Wind_Direction      

In [79]:
# Create a duration
df['Duration'] = df['End_Time'] - df['Start_Time']

In [80]:
# Convert to minutes
df['Duration'] = df['Duration']/np.timedelta64(1,'m')

In [59]:
# Does it correlate to Severity?
df['Duration'].corr(df['Severity'])

0.018434961609356315

In [9]:
# Does Distance correlate to Severity?
df['Distance(mi)'].corr(df['Severity'])

0.14639349572676352

In [10]:
df[['Start_Time', 'Start_Lat', 'Start_Lng', 'Airport_Code','Zipcode', 'TMC','Number','Precipitation(in)', 'Wind_Chill(F)', 'Wind_Speed(mph)', 'Weather_Condition','Weather_Timestamp']].head(10)

,Start_Time,Start_Lat,Start_Lng,Airport_Code,Zipcode,TMC,Number,Precipitation(in),Wind_Chill(F),Wind_Speed(mph),Weather_Condition,Weather_Timestamp
0,2016-02-08 05:46:00,39.865147,-84.058723,KFFO,45424,201.0,NaN,0.02,NaN,NaN,Light Rain,2016-02-08 05:58:00
1,2016-02-08 06:07:59,39.928059,-82.831184,KCMH,43068-3402,201.0,2584.0,0.00,NaN,NaN,Light Rain,2016-02-08 05:51:00
2,2016-02-08 06:49:27,39.063148,-84.032608,KI69,45176,201.0,NaN,NaN,33.3,3.5,Overcast,2016-02-08 06:56:00
3,2016-02-08 07:23:34,39.747753,-84.205582,KDAY,45417,201.0,NaN,NaN,31.0,4.6,Mostly Cloudy,2016-02-08 07:38:00
4,2016-02-08 07:39:07,39.627781,-84.188354,KMGY,45459,201.0,NaN,NaN,33.3,3.5,Mostly Cloudy,2016-02-08 07:53:00
5,2016-02-08 07:44:26,40.100590,-82.925194,KCMH,43081,201.0,NaN,0.03,35.5,3.5,Light Rain,2016-02-08 07:51:00
6,2016-02-08 07:59:35,39.758274,-84.230507,KDAY,45417-2476,201.0,376.0,NaN,31.0,3.5,Overcast,2016-02-08 07:56:00
7,2016-02-08 07:59:58,39.770382,-84.194901,KDAY,45405,201.0,NaN,NaN,31.0,3.5,Overcast,2016-02-08 07:56:00
8,2016-02-08 08:00:40,39.778061,-84.172005,KFFO,45404-1923,201.0,99.0,NaN,NaN,1.2,Mostly Cloudy,2016-02-08 07:58:00
9,2016-02-08 08:10:04,40.100590,-82.925194,KCMH,43081,201.0,NaN,0.02,33.8,4.6,Light Rain,2016-02-08 08:28:00


In [11]:
# % in TMC categories
df.TMC.value_counts()/2974335*100

201.0    63.544860
241.0     7.365377
245.0     1.207631
229.0     0.761616
203.0     0.564563
222.0     0.422010
244.0     0.375311
406.0     0.318290
246.0     0.213493
202.0     0.203306
343.0     0.195909
247.0     0.155194
236.0     0.071142
206.0     0.038899
248.0     0.034024
339.0     0.026023
341.0     0.016945
336.0     0.002723
200.0     0.002219
239.0     0.001816
351.0     0.000202
Name: TMC, dtype: float64

In [81]:
import requests

In [76]:
# http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=ea4b0cc4126041a4bb1153215202803&q=39.865147, -84.058723&format=json&date=2016-02-08&enddate=2016-04-08&includelocation=yes&tp=1

url = 'http://api.worldweatheronline.com/premium/v1/past-weather.ashx'
API_KEY = 'ea4b0cc4126041a4bb1153215202803'
loc = '39.865147, -84.058723'
d = '2016-04-08'
params = dict(key = API_KEY, q = loc, format = 'json', date = d, includelocation = 'yes', tp=1 )

res = requests.get(url, params)

#res = requests.get('http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=ea4b0cc4126041a4bb1153215202803&q=39.865147, -84.058723&format=json&date=2016-02-08&includelocation=yes&tp=12')



In [14]:
url = 'http://api.worldweatheronline.com/premium/v1/past-weather.ashx'
API_KEY = '*****'
loc = str(df['Start_Lat'][0]) + ', ' + str(df['Start_Lng'][0])
d = str(df.Start_Time[0].date())
params = dict(key = API_KEY, q = loc, format = 'json', date = d, includelocation = 'yes', tp=1 )

res = requests.get(url, params)

In [15]:
json = res.json()

In [105]:
# for i in df:  This errors out
for i in range(0, 2974334)
    if str(df.loc[i, 'Wind_Chill(F)']) == 'nan' or str(df.loc[i, 'Wind_Speed(mph)']) == 'nan' or str(df.loc[i, 'Precipitation(in)']) == 'nan':
            loc = str(df['Start_Lat'][i]) + ', ' + str(df['Start_Lng'][i])
            d = str(df.Start_Time[i].date())
            params = dict(key = API_KEY, q = loc, format = 'json', date = d, includelocation = 'yes', tp=1 )
            res = requests.get(url, params)
            json = res.json()
            if df.Start_Time[i].time().hour == 23:
                h = 23
            elif df.Start_Time[i].time().minute > 30:
                h = df.Start_Time[i].time().hour + 1
            else:
                h = df.Start_Time[i].time().hour

            if str(df.loc[i, 'Wind_Chill(F)']) == 'nan':
                df.loc[i,'Wind_Chill(F)'] = int(json['data']['weather'][0]['hourly'][h]['WindChillF'])
            if str(df.loc[i, 'Wind_Speed(mph)']) == 'nan':
                df.loc[i,'Wind_Speed(mph)'] = int(json['data']['weather'][0]['hourly'][h]['windspeedMiles'])
            if str(df.loc[i, 'Precipitation(in)']) == 'nan':
                df.loc[i,'Precipitation(in)'] = float(json['data']['weather'][0]['hourly'][h]['precipInches'])
    else:
        continue

KeyError: 'weather'

In [47]:
# Received a 429 error, too many requests
# Changed percentages of blanks just a hair

df[['Wind_Chill(F)', 'Wind_Speed(mph)', 'Precipitation(in)']].isna().sum()/2974335*100

Wind_Chill(F)        62.202677
Wind_Speed(mph)      14.812891
Precipitation(in)    67.093384
dtype: float64

In [104]:
# Row 2967 is now first line with NaNs
df.loc[2967, ['Wind_Chill(F)', 'Wind_Speed(mph)', 'Precipitation(in)']]

Wind_Chill(F)         NaN
Wind_Speed(mph)      10.4
Precipitation(in)     NaN
Name: 2967, dtype: object

In [ ]:
# RapidAPI for Dark Sky would allow unlimited records for $0.001.  67% of 3M records is 2M records and would cost $2K

In [ ]:
# Plan B:  Sort records by Zipcode, month, day, hour, then Fill Forward.
# This puts nearby regions together by month and day (over the years) and may 
# have a higher chance of experiencing similar weather patterns. 

In [82]:
df['Month'] = df['Start_Time'].dt.month

In [83]:
df['Day'] = df['Start_Time'].dt.day

In [84]:
df['Hour'] = df['Start_Time'].dt.hour

In [85]:
# Zipcode is null in only 0.03% of records, remove null zipcode rows
df = df[df['Zipcode'].notna()]

In [86]:
df['Zip2'] = df['Zipcode'].str[:2]

In [87]:
df.sort_values(by=['Zip2', 'Month', 'Day', 'Hour'], inplace = True)

In [68]:
df[['Start_Time','Airport_Code','Zip2','Precipitation(in)', 'Wind_Chill(F)', 'Wind_Speed(mph)', 'Weather_Condition','Weather_Timestamp']].tail(30)

,Start_Time,Airport_Code,Zip2,Precipitation(in),Wind_Chill(F),Wind_Speed(mph),Weather_Condition,Weather_Timestamp
2689901,2018-12-28 08:17:30,KSFF,99,0.01,14.0,15.0,Overcast,2018-12-28 07:53:00
308002,2016-12-28 09:52:02,KGEG,99,0.01,14.0,15.0,Scattered Clouds,2016-12-28 09:53:00
1988215,2017-12-28 09:41:10,KGEG,99,0.01,17.2,3.5,Light Snow,2017-12-28 09:33:00
2850983,2017-12-28 09:39:34,KGEG,99,0.01,17.2,3.5,Light Snow,2017-12-28 09:33:00
1988217,2017-12-28 10:57:28,KSFF,99,0.00,22.0,6.0,Light Snow,2017-12-28 11:00:00
1988220,2017-12-28 11:23:42,KSFF,99,0.00,22.0,6.0,Light Snow,2017-12-28 11:05:00
515827,2019-12-28 17:49:20,KSKA,99,0.00,22.0,6.0,N/A Precipitation,2019-12-28 17:20:00
308030,2016-12-28 19:13:24,KSFF,99,0.01,22.8,4.6,Clear,2016-12-28 18:53:00
1990414,2017-12-29 05:21:04,KSFF,99,0.01,26.5,3.5,Light Snow,2017-12-29 05:14:00
2851482,2017-12-29 08:49:19,KSFF,99,0.02,24.3,6.9,Light Snow,2017-12-29 08:53:00


In [88]:
# bfill just those three columns
col = ['Wind_Chill(F)', 'Wind_Speed(mph)', 'Precipitation(in)']
df.loc[:, col] = df.loc[:, col].bfill()

In [136]:
df2 = df[['Severity', 'Distance(mi)', 'Wind_Chill(F)', 'Wind_Speed(mph)', 'Precipitation(in)', 'Duration', 'Temperature(F)','Humidity(%)', 'Pressure(in)', 'Visibility(mi)']]

In [137]:
df2.dropna()

,Severity,Distance(mi),Wind_Chill(F),Wind_Speed(mph),Precipitation(in),Duration,Temperature(F),Humidity(%),Pressure(in),Visibility(mi)
1145129,3,0.00,-5.2,21.9,0.00,29.666667,50.0,89.0,29.53,10.0
1872873,3,0.00,-5.2,13.8,0.00,60.000000,10.6,37.0,30.27,10.0
1873534,3,0.00,-20.4,3.5,0.00,29.833333,-11.0,75.0,30.33,10.0
1145929,2,0.00,16.2,10.4,0.00,29.850000,26.1,51.0,30.30,10.0
1145993,2,0.00,-20.4,3.5,0.00,29.283333,24.1,55.0,30.29,10.0
...,...,...,...,...,...,...,...,...,...,...
519864,2,0.00,29.0,5.0,0.06,44.783333,33.0,92.0,28.03,1.0
519865,2,0.00,26.0,8.0,0.02,29.700000,33.0,89.0,27.54,1.0
519866,2,0.00,34.0,3.0,0.05,29.766667,34.0,92.0,28.01,2.0
519867,2,0.00,34.0,3.0,0.05,44.783333,34.0,92.0,28.01,2.0


In [174]:
df2.isna().sum()/2892508*100

Severity             0.000000
Distance(mi)         0.000000
Wind_Chill(F)        0.000000
Wind_Speed(mph)      0.000000
Precipitation(in)    0.000000
Duration             0.000000
Temperature(F)       1.907791
Humidity(%)          2.015310
Pressure(in)         1.633945
Visibility(mi)       2.240651
dtype: float64

In [175]:
# not all na dropped
# does not work:  df3 = df2[df2[['Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)']].notna()]

df2 = df2[df2['Temperature(F)'].notna()]
df2 = df2[df2['Humidity(%)'].notna()]
df2 = df2[df2['Pressure(in)'].notna()]
df2 = df2[df2['Visibility(mi)'].notna()]

In [177]:
df2.shape

(2892508, 10)

In [178]:
df2.isna().sum()/2892508*100

Severity             0.0
Distance(mi)         0.0
Wind_Chill(F)        0.0
Wind_Speed(mph)      0.0
Precipitation(in)    0.0
Duration             0.0
Temperature(F)       0.0
Humidity(%)          0.0
Pressure(in)         0.0
Visibility(mi)       0.0
dtype: float64

In [138]:
# remove outliers
from scipy import stats

In [179]:
z_scores = stats.zscore(df2)
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
new_df = df2[filtered_entries]

In [180]:
new_df.shape

(2663278, 10)